In [2]:
import os
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
correspondence = {}
correspondence['149Sm_CREB'] ='149Sm_pCREB'
correspondence['167Er_ERK'] ='167Er_pERK12'
correspondence['164Dy_IkB'] ='164Dy_IkB'
correspondence['159Tb_MAPKAPK2'] ='159Tb_pMAPKAPK2'
correspondence['166Er_NFkB'] ='166Er_pNFkB'
correspondence['151Eu_p38'] ='151Eu_pp38'
correspondence['155Gd_S6'] ='155Gd_pS6'
correspondence['153Eu_STAT1'] ='153Eu_pSTAT1'
correspondence['154Sm_STAT3'] ='154Sm_pSTAT3'
correspondence['150Nd_STAT5'] ='150Nd_pSTAT5'
correspondence['168Er_pSTAT6'] ='168Yb_pSTAT6'
correspondence['174Yb_HLADR'] ='174Yb_HLADR'
correspondence['169Tm_CD25'] ='169Tm_CD25'

In [27]:
def create_density_plots(dist_data, out_file, stimulated,marker,title_suffix=""):
    sns.set_theme(style="whitegrid")
    pts_sorted = sorted(dist_data.keys())

    num_plots = len(pts_sorted)
    cols = min(3, num_plots)
    rows = int(np.ceil(num_plots / cols))

    fig_width = max(5 * cols, 8)
    fig_height = 5 * rows
    fig, axes = plt.subplots(
        rows, cols, figsize=(fig_width, fig_height), constrained_layout=True
    )

    fig.suptitle(f"Correction on surge/perio for cMCs, LPS and {marker}", fontsize=16)

    if num_plots == 1:
        axes = np.array([axes])
    if stimulated:
        cat_labels = ["Stim corr surge", "Stim corr perio", "Stim uncorr surge", "Stim uncorr perio"]
    else:
        cat_labels = ["Unstim corr surge", "Unstim corr perio", "Unstim uncorr surge", "Unstim uncorr perio"]
    cat_colors = ["blue", "red", "green", "orange", "purple"]

    for i, (pt, ax) in enumerate(zip(pts_sorted, axes.flatten())):
        for label, color in zip(cat_labels, cat_colors):
            arr = dist_data[pt][label]
            if arr.size > 0:
                sns.kdeplot(
                    arr,
                    ax=ax,
                    label=f"{label} (n={arr.size})",
                    color=color,
                    fill=False,  # set tot True to fill the area under the curve
                    alpha=0.3,
                )

        ax.set_xlabel("Value", fontsize=12)
        ax.set_ylabel("Density", fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True)
    for j in range(i + 1, len(axes.flatten())):
        fig.delaxes(axes.flatten()[j])

    plt.savefig(out_file, dpi=200, bbox_inches="tight")
    plt.close()


def plot_result(corrected_path_surge,original_path_surge,corrected_path_perio,marker,original_path_perio,output_path_stim,output_path_unstim,doms_stim):
    corr_surge = ad.read(corrected_path_surge)
    corr_surge = corr_surge[:, correspondence[marker]].copy()
    stim_corr_surge = pd.Series(
        corr_surge[(corr_surge.obs["drug"] == doms_stim)].X.flatten(), name="Stim corr surge"
    )
    unstim_corr_surge = pd.Series(
        corr_surge[corr_surge.obs["drug"] == "Unstim"].X.flatten(), name="Unstim corr surge"
    )
    corr_perio= ad.read(corrected_path_perio)
    corr_perio = corr_perio[:, correspondence[marker]].copy()
    stim_corr_perio = pd.Series(
        corr_perio[(corr_perio.obs["drug"] == 'P. gingivalis')].X.flatten(), name="Stim corr perio"
    )
    unstim_corr_perio = pd.Series(
        corr_perio[corr_perio.obs["drug"] == "Unstim"].X.flatten(), name="Unstim corr perio"
    )
    
    uncorr_surge = ad.read(original_path_surge)
    uncorr_surge = uncorr_surge[:, correspondence[marker]].copy()
    stim_uncorr_surge = pd.Series(
        uncorr_surge[(uncorr_surge.obs["drug"] == doms_stim)].X.flatten(), name="Stim uncorr surge"
    )
    unstim_uncorr_surge = pd.Series(
        uncorr_surge[uncorr_surge.obs["drug"] == "Unstim"].X.flatten(), name="Unstim uncorr surge"
    )
    uncorr_perio= ad.read(original_path_perio)
    uncorr_perio = uncorr_perio[:, marker].copy()
    stim_uncorr_perio = pd.Series(
        uncorr_perio[(uncorr_perio.obs["drug"] == 'P. gingivalis')].X.flatten(), name="Stim uncorr perio"
    )
    unstim_uncorr_perio = pd.Series(
        uncorr_perio[uncorr_perio.obs["drug"] == "Unstim"].X.flatten(), name="Unstim uncorr perio"
    )
    dist_data_stim = {
        "Patient_1": {
            "Stim corr surge": stim_corr_surge.values,
            "Stim corr perio": stim_corr_perio.values,
            'Stim uncorr surge': stim_uncorr_surge.values,
            'Stim uncorr perio': stim_uncorr_perio.values
        }
    }
    dist_data_unstim = {
        "Patient_1": {
            "Unstim corr surge": unstim_corr_surge.values,
            "Unstim corr perio": unstim_corr_perio.values,
            'Unstim uncorr surge': unstim_uncorr_surge.values,
            'Unstim uncorr perio': unstim_uncorr_perio.values
        }
    }
    stimulated=True
    unstimulated=False
    create_density_plots(dist_data_stim, output_path_stim, stimulated,marker,title_suffix="")
    create_density_plots(dist_data_unstim, output_path_unstim, unstimulated,marker,title_suffix="")
    return

In [30]:
marker_list=correspondence.keys()
perio_stim_list_=['TNFa','P._gingivalis']
perio_cell_list_=['Granulocytes_(CD45-CD66+)','B-Cells_(CD19+CD3-)','Classical_Monocytes_(CD14+CD16-)','MDSCs_(lin-CD11b-CD14+HLADRlo)','mDCs_(CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate_Monocytes_(CD14+CD16+)','Non-classical_Monocytes_(CD14-CD16+)','CD56+CD16-_NK_Cells','CD56loCD16+NK_Cells','NK_Cells_(CD7+)','CD4_T-Cells','Tregs_(CD25+FoxP3+)','CD8_T-Cells','CD8-CD4-_T-Cells']
cell_type='B-Cells_(CD19+CD3-)'
stim='P._gingivalis'
if [stim, cell_type] not in [['P._gingivalis', 'Non-classical_Monocytes_(CD14-CD16+)'],['P._gingivalis', 'NK_Cells_(CD7+)'],['TNFa', 'Granulocytes_(CD45-CD66+)']]:
    for marker in marker_list:
        if stim=='P._gingivalis':
            doms_stim='LPS'
        else:
            doms_stim=stim
    
        corrected_path_surge = f"surge_on_perio_corrected_simple/surge_concatenated_LPS_Classical Monocytes (CD14+CD16-).h5ad"
        original_path_surge="surge_just_concat_train/surge_concatenated_LPS_Classical Monocytes (CD14+CD16-).h5ad"
        corrected_path_perio = 'perio_on_surge_corrected_simple/perio_data_sherlock_P. gingivalis_Classical Monocytes (CD14+CD16-).h5ad'
        original_path_perio='perio_just_concat/perio_data_sherlock_P. gingivalis_Classical Monocytes (CD14+CD16-).h5ad'
        output_path_stim = f"plot_simple_correction_on_unstim_unstim/{doms_stim}_{cell_type}_{marker}_surge.png"
        output_path_unstim = f"plot_simple_correction_on_unstim_stim/{doms_stim}_{cell_type}_{marker}_surge.png"
        
        plot_result(corrected_path_surge,original_path_surge,corrected_path_perio,marker,original_path_perio,output_path_stim,output_path_unstim,doms_stim)
        print(f"Plot {marker} for {cell_type} and {doms_stim}")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 149Sm_CREB for B-Cells_(CD19+CD3-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 167Er_ERK for B-Cells_(CD19+CD3-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


...

Plot 168Er_pSTAT6 for B-Cells_(CD19+CD3-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 174Yb_HLADR for B-Cells_(CD19+CD3-) and LPS


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Plot 169Tm_CD25 for B-Cells_(CD19+CD3-) and LPS
